# 随机完全区组设计

随机完全区组设计是一种完全区组设计，在完全区组设计中，全部试验按随机顺序进行就得到随机完全区组设计，简称为随机区组设计。随机化完全区组设计将整个试验环境划分成若干各自相对均匀的区组，然后把每一区组分成K个试验小区，随机布置K个处理的设计。由于每一区组必须而且也只能包括每一处理的一个小区，故称为完全区组。

## 举个栗子

说明：本栗子来自 林木育种学(NFU) 2023-2 学期 第 8 章遗传测定 第 11 页。该数据由边黎明老师提供。

设有一无性系比较试验，有 7 个无性系，完全随机区组设计，3 个重复，有个小区栽 3 株树，小区排列及各小区平均胸径如下：

![](https://zywu-blog-image.oss-cn-nanjing.aliyuncs.com/images/image-20230407101828467.png)

<center><b>表 1.</b> 完全随机区组设计中各小区的平均胸径</center>

表中的数值为该小区中3棵树的平均胸径，另外我们可以看到在3个区组中小区的排列是完全随机排列的。

首先，我们利用 `Python` 中来自 `pandas` 的 `DataFrame` 类的创建本次计算的原始数据集。

In [6]:
from IPython.display import HTML
import pandas as pd

data = {'Block': ['I', 'I', 'I', 'I', 'I', 'I', 'I',
                  'II', 'II', 'II', 'II', 'II', 'II', 'II',
                  'III', 'III', 'III', 'III', 'III', 'III', 'III'],
        'Plot': ['A6', 'A1', 'A5', 'A4', 'A3', 'A7', 'A2',
                 'A1', 'A4', 'A3', 'A6', 'A2', 'A5', 'A7',
                 'A6', 'A2', 'A7', 'A5', 'A3', 'A1', 'A4'],
        'Value':[20, 24, 22, 18, 21, 20, 20,
                 26, 16, 19, 21, 19, 20 ,19,
                 21, 21, 21, 23, 20, 23, 19]}
df = pd.DataFrame(data = data)

HTML(df.to_html(index=False))

Block,Plot,Value
I,A6,20
I,A1,24
I,A5,22
I,A4,18
I,A3,21
I,A7,20
I,A2,20
II,A1,26
II,A4,16
II,A3,19


### 表格整理

然后，我们需要将原始数据的表格进行整理，并计算各个小区在区组、各个区组在小区中的平均值。

首先利用 `pivot` 函数将长数据格式转换成短数据格式，并利用 `rename_axis` 函数移除 headers。

In [7]:
df_s = df.pivot(index='Plot', columns='Block', values='Value').rename_axis(None,axis=0).rename_axis(None,axis=1)
HTML(df_s.to_html())

,I,II,III
A1,24,26,23
A2,20,19,21
A3,21,19,20
A4,18,16,19
A5,22,20,23
A6,20,21,21
A7,20,19,21


通过 `pd.DataFrame` 类中 `mean` 函数计算每行每列平均值。

In [8]:
df_mean = df_s.copy()
df_mean['mean'] = df_mean.mean(numeric_only=True, axis=1).round(2)
df_mean.loc['mean'] = df_mean.mean(numeric_only=True, axis=0).round(2)
HTML(df_mean.to_html())

,I,II,III,mean
A1,24.00,26.0,23.00,24.33
A2,20.00,19.0,21.00,20.00
A3,21.00,19.0,20.00,20.00
A4,18.00,16.0,19.00,17.67
A5,22.00,20.0,23.00,21.67
A6,20.00,21.0,21.00,20.67
A7,20.00,19.0,21.00,20.00
mean,20.71,20.0,21.14,20.62


### 方差分解

$$
\bar{x} = \frac{1}{N} \sum_{i=1}^{m}\sum_{j=1}^{n} x_{ij}
$$
$$
SS_{T} = \sum_{i=1}^{m}\sum_{j=1}^{n}(x_{ij}-\bar{x})^{2}
$$
$$
SS_{t} = n \sum_{i=1}^{m}(\bar{x}_{i}-\bar{x})^{2}
$$
$$
SS_{R} = m \sum_{j=1}^{n}(\bar{x}_{j}-\bar{x})^{2}
$$
$$
SS_{e} = SS_{T} - SS_{t} - SS_{R}
$$
$$
MS = \frac{SS}{df}
$$
$$
F = \frac{MS_{t}}{MS_{e}}
$$

这里，$N$ 表示数据的总数，$m$ 表示小区数，$n$ 表示区组数，$SS_{T}$ 表示总变异，$SS_{t}$ 表示无性系间的变异，$SS_{R}$ 表示区组间的变异，$SS_{e}$ 表示试验误差。

### 方差分析表

我们通过上述公式可以计算得到结果，如下表所示：

In [107]:
anova_data = {'变异来源': ['无性系间', '区组间', '试验误差', '总变异'],
              'df': [6, 2, 12, 20],
              'SS': [74.28, 4.67, 16.00, 94.95],
              'MS': [12.38, 2.38, 1.33, ''],
              'F': ['9.29**', '', '', '']}
anova_df = pd.DataFrame(data = anova_data)
HTML(anova_df.to_html(index=False))

变异来源,df,SS,MS,F
无性系间,6,74.28,12.38,9.29**
区组间,2,4.67,2.38,
试验误差,12,16.00,1.33,
总变异,20,94.95,,


经过 F 检验，表明 7 个无性系的平均胸径间有显著差异，所以需要进行多重比较。

### 多重比较

1. 将 7 个无性系的平均胸径进行从高到低排序。

In [9]:
rank_data = {'Clone': ['A1', 'A5', 'A6', 'A2', 'A3', 'A7', 'A4'],
             'Value': [24.33, 21.67, 20.67, 20.00, 20.00, 20.00, 17.67],
             'Rank': [1, 2, 3, 4, 5, 6, 7]}
rank_df = pd.DataFrame(data = rank_data)
HTML(rank_df.to_html(index=False))

Clone,Value,Rank
A1,24.33,1
A5,21.67,2
A6,20.67,3
A2,20.00,4
A3,20.00,5
A7,20.00,6
A4,17.67,7


2. 计算标准误。

$$
\bar{s_{d}} = \sqrt{\frac{MS_{e}}{n}}
$$

根据上述公式可以计算出 $\bar{s_{d}}=0.665$。

3. 计算 $(t-1)$ 个无性系间最小显著范围。

$$
R_{p} = r_{p} \bar{s_{d}} \ \ for \ p=2,3,\cdots,t
$$

通过查[表](https://www.york.ac.uk/depts/maths/tables/duncan.pdf)可以得到不同显著水平下的$r_{p}$值。

In [10]:
rp_data = {'P': [2, 3, 4, 5, 6, 7],
           'r(12,0.01)': [4.32, 4.50, 4.62, 4.71, 4.77, 4.82],
           'r(12,0.05)': [3.08, 3.23, 3.31, 3.37, 3.41, 3.44]}
rp_df = pd.DataFrame(data = rp_data)
HTML(rp_df.to_html(index=False))

P,"r(12,0.01)","r(12,0.05)"
2,4.32,3.08
3,4.50,3.23
4,4.62,3.31
5,4.71,3.37
6,4.77,3.41
7,4.82,3.44


然后可以计算出 $R_p$:

In [12]:
import math

Rp_df = rp_df.copy()
Rp_df.iloc[:,1] = Rp_df.iloc[:,1] * 0.665
Rp_df.iloc[:,2] = Rp_df.iloc[:,2] * 0.665
Rp_df.iloc[:,1] = Rp_df.iloc[:,1].round(2)
Rp_df.iloc[:,2] = Rp_df.iloc[:,2].round(2)
HTML(Rp_df.to_html(index=False))

P,"r(12,0.01)","r(12,0.05)"
2,2.87,2.05
3,2.99,2.15
4,3.07,2.20
5,3.13,2.24
6,3.17,2.27
7,3.21,2.29


4. 最后根据显著边界值判断各个无性系之间的差异。

利用 `Python` 根据上述过程进行计算。

In [1]:
""" 
Duncan's new multiple range test
-----------------------------------
This is a simple implementation of Duncan's new multiple range test for multiple comparison.

Author: Zell Wu
Date: 4/7/2023
"""

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import string


def dmrt(values, sd, rp, no_digits=2):
    """ Duncan's new multiple range test
    Input:
        values: The values from different treatments (do not need to be sorted).
        sd: The standard error of the mean difference.
        rp: The values of significant studentized ranges obtained from table.
        no_digits: The number of digits after the dot.
    Output:
        A table with MRT alphabetical notation.
    """
    # set critical values
    Rp = [round(i * sd, no_digits) for i in rp]
    # sort
    sorted_values = sorted(values, reverse=True)
    # comparison
    res = []
    done = False
    for i in range(0, len(sorted_values)-1):
        # loop quit if we has searched all elements
        if done:
            break
        temp = [i+1]
        # loop
        for j in range(i+1, len(sorted_values)):
            # Calculate the difference between two values
            diff = sorted_values[i] - sorted_values[j]
            rp = Rp[j-i-1]
            # Compare difference to critical value
            if diff <= rp:
                # j+1 is no significant different with i+1 
                temp.append(j+1)
                # there are no significant differences between i+1 and the end
                if j == len(sorted_values) - 1:
                    res.append(temp)
                    done = True
                    break
            else:
                # duplication e.g. [3,4,5] [4,5]
                if res and i != len(sorted_values) - 2 and j == res[-1][-1]:
                    break
                # comparison between the last two elements, so add the last element into the end individually
                if i == len(sorted_values) - 2 and j == len(sorted_values) - 1:
                    res.append([j+1])
                    done = True
                    break
                # normal condition: there are significant differences between i+1 and j+1
                res.append(temp)
                break
    return res


if __name__ == "__main__":

    values = [24.33, 20.00, 20.00, 17.67, 21.67, 20.67, 20.00]
    sd = 0.665
    # You can change the p_value below by commenting and uncommenting
    rp = [4.320, 4.504, 4.622, 4.705, 4.767, 4.815]   # p_value=0.01
    #rp = [3.081, 3.225, 3.312, 3.370, 3.410, 3.439]   # p_value=0.05

    res = dmrt(values, sd, rp)
    print(res)

    show_df = pd.DataFrame(data = {'Clone': ['A1', 'A5', 'A6', 'A2', 'A3', 'A7', 'A4'],
                                   'Mean': [24.33, 21.67, 20.67, 20.00, 20.00, 20.00, 17.67]})
    ordinal = 0
    for lst in res:
        show_df[ordinal] = [''] * len(values)
        for i in lst:
            show_df[ordinal][i-1] = '|'
        ordinal += 1
    show_df['Group'] = [''] * len(values)

    for i in range(show_df.shape[0]):
        for j in range(ordinal):
            if show_df[j][i] == '|':
                show_df['Group'][i] += list(string.ascii_lowercase)[j]
                
    display(show_df)

[[1, 2], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7]]


,Clone,Mean,0,1,2,Group
0,A1,24.33,|,,,a
1,A5,21.67,|,|,,ab
2,A6,20.67,,|,|,bc
3,A2,20.00,,|,|,bc
4,A3,20.00,,|,|,bc
5,A7,20.00,,|,|,bc
6,A4,17.67,,,|,c


## 参考资料

> https://baike.baidu.com/item/%E9%9A%8F%E6%9C%BA%E5%AE%8C%E5%85%A8%E5%8C%BA%E7%BB%84%E8%AE%BE%E8%AE%A1/19196063 \
> https://en.wikipedia.org/wiki/Duncan%27s_new_multiple_range_test \
> https://www.biologydiscussion.com/vegetable-breeding-duncans-multiple-range-test-with-diagram-statistics/68180 \
> https://www.york.ac.uk/depts/maths/tables/duncan.pdf \
> http://www.sthda.com/english/wiki/two-way-anova-test-in-r \
> https://rdrr.io/cran/agricolae/man/duncan.test.html